In [6]:
from huggingface_hub import notebook_login
notebook_login()

**Basic**

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW
from transformers import pipeline
import torch

model_id = "google/gemma-3-1b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

C:\laragon\bin\python\python-3.10\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JAHID\.cache\huggingface\hub\models--google--gemma-3-1b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [19]:
generation_pipeline = pipeline(task="text-generation",model=model,tokenizer=tokenizer)

generation_pipeline("Hello how are you?",max_new_tokens=25)

Device set to use cpu


[{'generated_text': "Hello how are you?\n\nI'm trying to figure out the best way to handle a potential conflict with a colleague.\n\nWe've been"}]

**Batch Generation**

In [20]:
generation_pipeline([
    "Hello what are you?",
    "The capital of Bangladesh is"
],max_new_tokens=10)

[[{'generated_text': 'Hello what are you?\n\nIt is difficult to know what you are.'}],
 [{'generated_text': 'The capital of Bangladesh is Dhaka. It is a bustling city with a rich'}]]

In [22]:
tokenizer.pad_token = tokenizer.eos_token

**Tokenizer**

In [24]:
input_prompt = [
    "Hello how are you doing tell me?",
    "The capital of Bangladesh is"
]
tokenized = tokenizer(input_prompt,padding=True,return_tensors="pt")
print(tokenized["input_ids"].shape)

torch.Size([2, 9])


In [26]:
tokenized["input_ids"]

tensor([[     2,   9259,   1217,    659,    611,   3490,   3442,    786, 236881],
        [     1,      1,      1,      2,    818,   5279,    529,  33752,    563]])

In [27]:
tokenizer.batch_decode(tokenized["input_ids"])

['<bos>Hello how are you doing tell me?',
 '<eos><eos><eos><bos>The capital of Bangladesh is']

**Chat Templates**

In [45]:
prompt = [
    {
        "role": "system",
        "content": "You are helpful AI Assistant"
    },
    {
        "role":"user",
        "content":"Where to study in Machine Learning in Dhaka"
    },
    {
        "role":"assistant",
        "content":"Study at"
    }
]
tokenizer.pad_token = tokenizer.eos_token

tokenized = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    contine_final_message=True,
    tokenize=True,
    padding=True,
    return_tensors="pt"
)
print(tokenized)

tensor([[     2,    105,   2364,    107,   3048,    659,  11045,  12498,  22083,
            108,  10936,    531,   2748,    528,  15313,  19180,    528, 111425,
            106,    107,    105,   4368,    107,  51462,    657,    106,    107,
            105,   4368,    107]])


In [49]:
out = model.generate(tokenized,max_new_tokens=100)

In [47]:
out

tensor([[     2,    105,   2364,    107,   3048,    659,  11045,  12498,  22083,
            108,  10936,    531,   2748,    528,  15313,  19180,    528, 111425,
            106,    107,    105,   4368,    107,  51462,    657,    106,    107,
            105,   4368,    107,  19058, 236764,   1531, 236789, 236751,   2541,
           1679,   1298,    611,    740,   2748,  15313,  19180,    528, 111425,
         236761,   1030, 236858, 236751,    496]])

In [50]:
decoded = tokenizer.batch_decode(out)
print(decoded[0])

<bos><start_of_turn>user
You are helpful AI Assistant

Where to study in Machine Learning in Dhaka<end_of_turn>
<start_of_turn>model
Study at<end_of_turn>
<start_of_turn>model
Okay, let's break down where you can study Machine Learning in Dhaka. There are several excellent options, ranging from formal institutions to online platforms. Here’s a breakdown, categorized by type and with estimated costs (which can fluctuate):

**1. Formal Universities & Colleges (Highly Recommended)**

* **SRJ University:** (Highly Recommended - Often considered the top choice)
    * **Program:** Master of Science in Data Science and Machine Learning (MSDSML)
    *


**Lora**

In [53]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=32,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['q_proj','v_proj']
)

model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 2,981,888 || all params: 1,002,867,840 || trainable%: 0.2973


In [55]:
training_prompt = [
    {
        "role":"user",
        "content":"Where to study in Machine Learning in Dhaka?"
    },
    {
        "role":"assistant",
        "content":"Study at"
    }
]
target_response = "Interactive Care"

In [56]:
test_tokenized = tokenizer.apply_chat_template(training_prompt,continue_final_message=True,return_tensors="pt")
test_out = model.generate(test_tokenized,max_new_tokens=10)
print(tokenizer.batch_decode(test_out,skip_special_tokens=True)[0])

user
Where to study in Machine Learning in Dhaka?
model
Study at Machine Learning in Dhaka is a bit tricky because it


In [75]:
def generate_input_output_pair(prompt, target_response):
    chat_templates = tokenizer.apply_chat_template(prompt,continue_final_message=True,tokenize=False)
    full_response_text = [
        (chat_template + " " + target_response + tokenizer.eos_token)
        for chat_template, target_response in zip(chat_templates,target_response)
    ]
    input_ids_tokenized = tokenizer(full_response_text,return_tensors="pt",add_special_tokens=False)["input_ids"]

    labels_tokenized = tokenizer([" "+response+tokenizer.eos_token for response in target_response],
                               add_special_tokens=False,return_tensors="pt",padding="max_length",max_length=input_ids_tokenized.shape[1])["input_ids"]

    labels_tokenized_fixed = torch.where(labels_tokenized != tokenizer.pad_token_ids,labels_tokenized,-100)
    labels_tokenized_fixed[:,-1] = tokenizer.pad_token_id

    input_ids_tokenized_left_shifted= input_ids_tokenized[:,:-1]
    input_ids_tokenized_right_shifted= input_ids_tokenized[:,1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id

    return {
        "input_ids": input_ids_tokenized_left_shifted,
        "attention_mask": input_ids_tokenized_left_shifted,
        "labels": input_ids_tokenized_left_shifted,
    }

In [58]:
import torch.nn as nn

def calculate_loss(logits, labels):
    loss_fn = nn.CrossEntropyLoss(reduction='none')
    cross_entropy_loss = loss_fn(logits.view(-1,logits.size(-1)),labels.view(-1))
    return cross_entropy_loss

In [82]:
data = generate_input_output_pair(prompt=[training_prompt],target_response=[target_response])
data["input_ids"] = data["input_ids"]
data["labels"] = data["labels"]

optimizer = AdamW(model.parameters(),lr=1e-3,weight_decay=0.01)

for _ in range(20):
    out = model(input_ids=data["input_ids"])
    loss = calculate_loss(out.logits,data["labels"]).mean()

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print("loss: ",loss.item())

loss:  1.633334755897522
loss:  1.2560653686523438
loss:  0.7621409893035889
loss:  0.5239904522895813
loss:  0.4461246728897095
loss:  0.456743061542511
loss:  0.4348416328430176
loss:  0.4238668382167816
loss:  0.4177795350551605
loss:  0.41063806414604187
loss:  0.40056726336479187
loss:  0.39028969407081604
loss:  0.38052016496658325
loss:  0.3710635304450989
loss:  0.35946932435035706
loss:  0.3355048596858978
loss:  0.3184804916381836
loss:  0.3028225302696228
loss:  0.2771447002887726
loss:  0.24802570044994354


In [83]:
test_tokenized = tokenizer.apply_chat_template(training_prompt,continue_final_message=True,return_tensors="pt")
test_out = model.generate(test_tokenized,max_new_tokens=100)
print(tokenizer.batch_decode(test_out,skip_special_tokens=True)[0])

user
Where to study in Machine Learning in Dhaka?
model
Study at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at at
